# Create snapshots from Archived data

Procedure:
1. Create a new subfolder for the wanted ion species, e.g. Xe
2. Copy the fetch.sh to the new folder
3. Edit the `from` `to` timestamps
4. Run the fetch.sh script to get three data files for ND, IC and HMR
5. Start this notebook
6. Edit the `data_folder` variable
7. Run the notebook

Maintain the *_pv.txt list:
- Comment out the PVs which are not being archived
- Uncomment out the PVs which are being archived

In [1]:
import pandas as pd
from datetime import datetime
from getpass import getuser
from phantasy_apps.threshold_manager._model import _get_dataframe_
from phantasy_apps.threshold_manager.tools import make_snapshot

In [2]:
def get_data_dict(row: pd.Series):
    """Return a dict of data from the input row of data.
    """
    data_list = []
    
    # Add timestamp
    data_list.append(('timestamp', datetime.strptime(row.name, '%Y-%m-%d %H:%M:%S%z').timestamp()))
    
    # Add user
    data_list.append(('user', getuser()))
    
    # Add ion info: ion source name, element name, atomic number, mass, charge, charge state
    def get_isrc_name(i: int):
        return {1: 'ISRC1', 2: 'ISRC2'}.get(i, 'ISRC1')

    def get_ion_name(a: int):
        return {124: 'Xe', 36: 'Ar', 64: 'Zn', 48: 'Ca'}.get(a, 'Unknown')

    def get_ion_number(a: int):
        return {124: 54, 36: 18, 64: 30, 48: 20}.get(a, 'Unknown')
    
    isrc_name = get_isrc_name(int(row['ACS_DIAG:DEST:ACTIVE_ION_SOURCE']))
    
    if isrc_name == 'ISRC2':
        ion_mass = int(row['FE_ISRC2:BEAM:A_BOOK'])
        ion_charge = int(row['FE_ISRC2:BEAM:Q_BOOK'])
    else:
        ion_mass = int(row['FE_ISRC1:BEAM:A_BOOK'])
        ion_charge = int(row['FE_ISRC1:BEAM:A_BOOK'])
    
    data_list.append(('isrc_name', isrc_name))
    data_list.append(('ion_name', get_ion_name(ion_mass)))
    data_list.append(('ion_number', get_ion_number(ion_mass)))
    data_list.append(('ion_mass', ion_mass))
    data_list.append(('ion_charge', ion_charge))
    data_list.append(('ion_charge_state', row['ACC_OPS:BEAM:Q_STRIP']))
    
    # Add beam power, beam energy, beam bound, beam destination
    def get_bound_name(i: int):
        return {0: 'LINAC', 1: 'FSEE'}.get(i, 0)
    
    data_list.append(('power', row['ACS_DIAG:BPM_ENGY4:POWER_RD']))
    data_list.append(('energy', row['ACS_DIAG:BPM_ENGY4:KE_RD']))
    data_list.append(('bound', get_bound_name(row.get('ACS_DIAG:DEST:FSEE_LINE_RD', 0))))
    
    # Add diag data
    data_list.extend(list(row.items()))
    
    return dict(data_list)

## Open the database

In [3]:
import sqlite3
con = sqlite3.connect("/files/shared/ap/settings_manager/mps_tm.db")

## Data Folder

In [60]:
data_folder = "./Ca"

## ND

In [61]:
df = pd.read_csv(f'{data_folder}/data_nd_1s.csv', index_col='time')

In [62]:
df

,FE_ISRC1:BEAM:A_BOOK,FE_ISRC1:BEAM:Z_BOOK,FE_ISRC1:BEAM:Q_BOOK,FE_ISRC2:BEAM:A_BOOK,FE_ISRC2:BEAM:Q_BOOK,ACC_OPS:BEAM:Q_STRIP,ACS_DIAG:BPM_ENGY4:KE_RD,ACS_DIAG:BPM_ENGY4:POWER_RD,ACS_DIAG:DEST:ACTIVE_ION_SOURCE,FE_RFQ:ND_D1025:LAVG_RD,...,LS2_CD12:ND_D3872:TRIP_LAVGLOEN_CMD,LS2_CD12:ND_D3872:TRIP_EN_CMD,LS2_CD12:ND_D3872:TRIP_10MHI_CSET,LS2_CD12:ND_D3872:TRIP_10MLO_CSET,LS2_CD12:ND_D3872:TRIP_1500UHI_CSET,LS2_CD12:ND_D3872:TRIP_1500ULO_CSET,LS2_CD12:ND_D3872:TRIP_150UHI_CSET,LS2_CD12:ND_D3872:TRIP_150ULO_CSET,LS2_CD12:ND_D3872:TRIP_15UHI_CSET,LS2_CD12:ND_D3872:TRIP_15ULO_CSET
time,,,,,,,,,,,,,,,,,,,,,
2023-07-11 19:37:01-04:00,86.0,36.0,19.0,48.0,10.0,34.0,216.702563,4613.409012,2.0,0.000050,...,1.0,80.0,0.0,-14.0,0.0,-93.0,0.0,-0.0,0.0,-0.0
2023-07-11 19:37:02-04:00,86.0,36.0,19.0,48.0,10.0,34.0,216.702842,4613.409012,2.0,0.000071,...,1.0,80.0,0.0,-14.0,0.0,-93.0,0.0,-0.0,0.0,-0.0
2023-07-11 19:37:03-04:00,86.0,36.0,19.0,48.0,10.0,34.0,216.703789,4613.409012,2.0,0.000082,...,1.0,80.0,0.0,-14.0,0.0,-93.0,0.0,-0.0,0.0,-0.0
2023-07-11 19:37:04-04:00,86.0,36.0,19.0,48.0,10.0,34.0,216.703789,4613.409012,2.0,0.000041,...,1.0,80.0,0.0,-14.0,0.0,-93.0,0.0,-0.0,0.0,-0.0
2023-07-11 19:37:05-04:00,86.0,36.0,19.0,48.0,10.0,34.0,216.703789,4620.876668,2.0,0.000021,...,1.0,80.0,0.0,-14.0,0.0,-93.0,0.0,-0.0,0.0,-0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-07-11 20:36:55-04:00,86.0,36.0,19.0,48.0,10.0,34.0,0.000000,0.000000,2.0,0.000120,...,1.0,80.0,0.0,-14.0,0.0,-93.0,0.0,-0.0,0.0,-0.0
2023-07-11 20:36:56-04:00,86.0,36.0,19.0,48.0,10.0,34.0,0.000000,0.000000,2.0,0.000115,...,1.0,80.0,0.0,-14.0,0.0,-93.0,0.0,-0.0,0.0,-0.0
2023-07-11 20:36:57-04:00,86.0,36.0,19.0,48.0,10.0,34.0,0.000000,0.000000,2.0,0.000130,...,1.0,80.0,0.0,-14.0,0.0,-93.0,0.0,-0.0,0.0,-0.0


In [63]:
# select a row (first hit) with max power value
max_power = df['ACS_DIAG:BPM_ENGY4:POWER_RD'].max()

In [64]:
max_power

4620.876667548304

In [65]:
df[df['ACS_DIAG:BPM_ENGY4:POWER_RD'] == max_power]

,FE_ISRC1:BEAM:A_BOOK,FE_ISRC1:BEAM:Z_BOOK,FE_ISRC1:BEAM:Q_BOOK,FE_ISRC2:BEAM:A_BOOK,FE_ISRC2:BEAM:Q_BOOK,ACC_OPS:BEAM:Q_STRIP,ACS_DIAG:BPM_ENGY4:KE_RD,ACS_DIAG:BPM_ENGY4:POWER_RD,ACS_DIAG:DEST:ACTIVE_ION_SOURCE,FE_RFQ:ND_D1025:LAVG_RD,...,LS2_CD12:ND_D3872:TRIP_LAVGLOEN_CMD,LS2_CD12:ND_D3872:TRIP_EN_CMD,LS2_CD12:ND_D3872:TRIP_10MHI_CSET,LS2_CD12:ND_D3872:TRIP_10MLO_CSET,LS2_CD12:ND_D3872:TRIP_1500UHI_CSET,LS2_CD12:ND_D3872:TRIP_1500ULO_CSET,LS2_CD12:ND_D3872:TRIP_150UHI_CSET,LS2_CD12:ND_D3872:TRIP_150ULO_CSET,LS2_CD12:ND_D3872:TRIP_15UHI_CSET,LS2_CD12:ND_D3872:TRIP_15ULO_CSET
time,,,,,,,,,,,,,,,,,,,,,
2023-07-11 19:37:05-04:00,86.0,36.0,19.0,48.0,10.0,34.0,216.703789,4620.876668,2.0,0.000021,...,1.0,80.0,0.0,-14.0,0.0,-93.0,0.0,-0.0,0.0,-0.0
2023-07-11 19:37:06-04:00,86.0,36.0,19.0,48.0,10.0,34.0,216.703789,4620.876668,2.0,0.000019,...,1.0,80.0,0.0,-14.0,0.0,-93.0,0.0,-0.0,0.0,-0.0
2023-07-11 19:37:07-04:00,86.0,36.0,19.0,48.0,10.0,34.0,216.703789,4620.876668,2.0,0.000009,...,1.0,80.0,0.0,-14.0,0.0,-93.0,0.0,-0.0,0.0,-0.0
2023-07-11 19:37:08-04:00,86.0,36.0,19.0,48.0,10.0,34.0,216.703789,4620.876668,2.0,0.000035,...,1.0,80.0,0.0,-14.0,0.0,-93.0,0.0,-0.0,0.0,-0.0
2023-07-11 19:37:09-04:00,86.0,36.0,19.0,48.0,10.0,34.0,216.703789,4620.876668,2.0,0.000045,...,1.0,80.0,0.0,-14.0,0.0,-93.0,0.0,-0.0,0.0,-0.0
2023-07-11 19:37:10-04:00,86.0,36.0,19.0,48.0,10.0,34.0,216.703789,4620.876668,2.0,0.000070,...,1.0,80.0,0.0,-14.0,0.0,-93.0,0.0,-0.0,0.0,-0.0
2023-07-11 19:37:11-04:00,86.0,36.0,19.0,48.0,10.0,34.0,216.705959,4620.876668,2.0,0.000075,...,1.0,80.0,0.0,-14.0,0.0,-93.0,0.0,-0.0,0.0,-0.0
2023-07-11 19:37:12-04:00,86.0,36.0,19.0,48.0,10.0,34.0,216.708614,4620.876668,2.0,0.000067,...,1.0,80.0,0.0,-14.0,0.0,-93.0,0.0,-0.0,0.0,-0.0
2023-07-11 19:37:13-04:00,86.0,36.0,19.0,48.0,10.0,34.0,216.706613,4620.876668,2.0,0.000089,...,1.0,80.0,0.0,-14.0,0.0,-93.0,0.0,-0.0,0.0,-0.0


In [66]:
row1 = df[df['ACS_DIAG:BPM_ENGY4:POWER_RD'] == max_power].iloc[0]

In [67]:
row1.name

'2023-07-11 19:37:05-04:00'

In [68]:
data_dict = get_data_dict(row1)

In [69]:
# build diag data snapshot from data_list per device type

In [70]:
df_nd = _get_dataframe_('ND', data_dict)

In [71]:
df_nd;

In [72]:
# add the generated snapshot to the database

In [73]:
make_snapshot(['ND'], data_dict, f"Generated with archived data on {datetime.now().isoformat()[:-7]}",
              ['AA', 'ND'], con)

## IC

In [74]:
df = pd.read_csv(f'{data_folder}/data_ic_1s.csv', index_col='time')

In [75]:
df

,FE_ISRC1:BEAM:A_BOOK,FE_ISRC1:BEAM:Z_BOOK,FE_ISRC1:BEAM:Q_BOOK,FE_ISRC2:BEAM:A_BOOK,FE_ISRC2:BEAM:Q_BOOK,ACC_OPS:BEAM:Q_STRIP,ACS_DIAG:BPM_ENGY4:KE_RD,ACS_DIAG:BPM_ENGY4:POWER_RD,ACS_DIAG:DEST:ACTIVE_ION_SOURCE,FS1_STRL:IC_D2227:LAVG_RD,...,BDS_FFS:IC_D5800:TRIP_LAVGLOEN_CMD,BDS_FFS:IC_D5800:TRIP_EN_CMD,BDS_FFS:IC_D5800:TRIP_10MHI_CSET,BDS_FFS:IC_D5800:TRIP_10MLO_CSET,BDS_FFS:IC_D5800:TRIP_1500UHI_CSET,BDS_FFS:IC_D5800:TRIP_1500ULO_CSET,BDS_FFS:IC_D5800:TRIP_150UHI_CSET,BDS_FFS:IC_D5800:TRIP_150ULO_CSET,BDS_FFS:IC_D5800:TRIP_15UHI_CSET,BDS_FFS:IC_D5800:TRIP_15ULO_CSET
time,,,,,,,,,,,,,,,,,,,,,
2023-07-11 19:37:01-04:00,86.0,36.0,19.0,48.0,10.0,34.0,216.702563,4613.409012,2.0,-0.000011,...,1.0,80.0,0.0,-0.0056,0.0,-0.037,0.0,0.0,0.0,0.0
2023-07-11 19:37:02-04:00,86.0,36.0,19.0,48.0,10.0,34.0,216.702842,4613.409012,2.0,-0.000010,...,1.0,80.0,0.0,-0.0056,0.0,-0.037,0.0,0.0,0.0,0.0
2023-07-11 19:37:03-04:00,86.0,36.0,19.0,48.0,10.0,34.0,216.703789,4613.409012,2.0,-0.000010,...,1.0,80.0,0.0,-0.0056,0.0,-0.037,0.0,0.0,0.0,0.0
2023-07-11 19:37:04-04:00,86.0,36.0,19.0,48.0,10.0,34.0,216.703789,4613.409012,2.0,-0.000010,...,1.0,80.0,0.0,-0.0056,0.0,-0.037,0.0,0.0,0.0,0.0
2023-07-11 19:37:05-04:00,86.0,36.0,19.0,48.0,10.0,34.0,216.703789,4620.876668,2.0,-0.000010,...,1.0,80.0,0.0,-0.0056,0.0,-0.037,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-07-11 20:36:55-04:00,86.0,36.0,19.0,48.0,10.0,34.0,0.000000,0.000000,2.0,-0.000010,...,1.0,80.0,0.0,-0.0056,0.0,-0.037,0.0,0.0,0.0,0.0
2023-07-11 20:36:56-04:00,86.0,36.0,19.0,48.0,10.0,34.0,0.000000,0.000000,2.0,-0.000010,...,1.0,80.0,0.0,-0.0056,0.0,-0.037,0.0,0.0,0.0,0.0
2023-07-11 20:36:57-04:00,86.0,36.0,19.0,48.0,10.0,34.0,0.000000,0.000000,2.0,-0.000010,...,1.0,80.0,0.0,-0.0056,0.0,-0.037,0.0,0.0,0.0,0.0


In [76]:
# select a row (first hit) with max power value
max_power = df['ACS_DIAG:BPM_ENGY4:POWER_RD'].max()
row1 = df[df['ACS_DIAG:BPM_ENGY4:POWER_RD'] == max_power].iloc[4]
data_dict = get_data_dict(row1)
df_ic = _get_dataframe_('IC', data_dict)

In [77]:
df_ic

,Device,Average,Peak Avg.,1 MHz Std.,Avg. Time,LongAvg.Th(H),LongAvg.Th(H)?,LongAvg.Th(L),LongAvg.Th(L)?,10ms Th(H),...,1500us Th(L),1500us Th(L)?,150us Th(H),150us Th(H)?,150us Th(L),150us Th(L)?,15us Th(H),15us Th(H)?,15us Th(L),15us Th(L)?
0,FS1_STRL:IC_D2227,-9.239387e-06,-1.002056e-05,0.012349,1.0,0.0,0.0,-0.000560,1.0,0.0,...,-0.0373,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,FS1_STRL:IC_D2228,-3.383181e-06,-3.117789e-06,0.009053,1.0,0.0,0.0,-0.000595,1.0,0.0,...,-0.0370,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,FS1_STRL:IC_D2246,-9.612151e-05,-9.625877e-05,0.009789,1.0,0.0,0.0,-0.000865,1.0,0.0,...,-0.0370,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,FS1_STRL:IC_D2247,-1.058492e-04,-1.054255e-04,0.035477,1.0,0.0,0.0,-0.000913,1.0,0.0,...,-0.0370,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,FS1_BTS:IC_D2478,-1.596126e-05,-1.693890e-05,0.009392,1.0,0.0,0.0,-0.000655,1.0,0.0,...,-0.0370,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,FS1_BTS:IC_D2479,-2.477965e-05,-2.818328e-05,0.016559,1.0,0.0,0.0,-0.000705,1.0,0.0,...,-0.0370,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,FS1_BTS:IC_D2498,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,FS1_BTS:IC_D2499,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,FS2_BTS:IC_D3940,-5.941924e-06,-6.059690e-06,0.005121,1.0,0.0,0.0,-0.000560,1.0,0.0,...,-0.0370,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,FS2_BBS:IC_D4018,-1.036589e-06,-1.923459e-06,0.002822,1.0,0.0,0.0,-0.003000,1.0,0.0,...,-0.0370,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [78]:
make_snapshot(['IC'], data_dict, f"Generated with archived data on {datetime.now().isoformat()[:-7]}",
              ['AA', 'IC'], con)

## HMR

In [79]:
df = pd.read_csv(f'{data_folder}/data_hmr_1s.csv', index_col='time')

In [80]:
df

,FE_ISRC1:BEAM:A_BOOK,FE_ISRC1:BEAM:Z_BOOK,FE_ISRC1:BEAM:Q_BOOK,FE_ISRC2:BEAM:A_BOOK,FE_ISRC2:BEAM:Q_BOOK,ACC_OPS:BEAM:Q_STRIP,ACS_DIAG:BPM_ENGY4:KE_RD,ACS_DIAG:BPM_ENGY4:POWER_RD,ACS_DIAG:DEST:ACTIVE_ION_SOURCE,LS1_WA01:HMR_D1156:LAVG_RD,...,LS3_WD06:HMR_D4700:TRIP_LAVGLOEN_CMD,LS3_WD06:HMR_D4700:TRIP_EN_CMD,LS3_WD06:HMR_D4700:TRIP_10MHI_CSET,LS3_WD06:HMR_D4700:TRIP_10MLO_CSET,LS3_WD06:HMR_D4700:TRIP_1500UHI_CSET,LS3_WD06:HMR_D4700:TRIP_1500ULO_CSET,LS3_WD06:HMR_D4700:TRIP_150UHI_CSET,LS3_WD06:HMR_D4700:TRIP_150ULO_CSET,LS3_WD06:HMR_D4700:TRIP_15UHI_CSET,LS3_WD06:HMR_D4700:TRIP_15ULO_CSET
time,,,,,,,,,,,,,,,,,,,,,
2023-07-11 19:37:01-04:00,86.0,36.0,19.0,48.0,10.0,34.0,216.702563,4613.409012,2.0,-0.000028,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-07-11 19:37:02-04:00,86.0,36.0,19.0,48.0,10.0,34.0,216.702842,4613.409012,2.0,-0.000028,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-07-11 19:37:03-04:00,86.0,36.0,19.0,48.0,10.0,34.0,216.703789,4613.409012,2.0,-0.000027,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-07-11 19:37:04-04:00,86.0,36.0,19.0,48.0,10.0,34.0,216.703789,4613.409012,2.0,-0.000028,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-07-11 19:37:05-04:00,86.0,36.0,19.0,48.0,10.0,34.0,216.703789,4620.876668,2.0,-0.000027,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-07-11 20:36:55-04:00,86.0,36.0,19.0,48.0,10.0,34.0,0.000000,0.000000,2.0,-0.000029,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-07-11 20:36:56-04:00,86.0,36.0,19.0,48.0,10.0,34.0,0.000000,0.000000,2.0,-0.000028,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-07-11 20:36:57-04:00,86.0,36.0,19.0,48.0,10.0,34.0,0.000000,0.000000,2.0,-0.000027,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [81]:
# select a row (first hit) with max power value
max_power = df['ACS_DIAG:BPM_ENGY4:POWER_RD'].max()
row1 = df[df['ACS_DIAG:BPM_ENGY4:POWER_RD'] == max_power].iloc[6]
data_dict = get_data_dict(row1)
df_hmr = _get_dataframe_('HMR', data_dict)

In [82]:
data_dict

{'timestamp': 1689118631.0,
 'user': 'zhangt',
 'isrc_name': 'ISRC2',
 'ion_name': 'Ca',
 'ion_number': 20,
 'ion_mass': 48,
 'ion_charge': 10,
 'ion_charge_state': 34.0,
 'power': 4620.876667548304,
 'energy': 216.70595884497985,
 'bound': 'LINAC',
 'FE_ISRC1:BEAM:A_BOOK': 86.0,
 'FE_ISRC1:BEAM:Z_BOOK': 36.0,
 'FE_ISRC1:BEAM:Q_BOOK': 19.0,
 'FE_ISRC2:BEAM:A_BOOK': 48.0,
 'FE_ISRC2:BEAM:Q_BOOK': 10.0,
 'ACC_OPS:BEAM:Q_STRIP': 34.0,
 'ACS_DIAG:BPM_ENGY4:KE_RD': 216.70595884497985,
 'ACS_DIAG:BPM_ENGY4:POWER_RD': 4620.876667548304,
 'ACS_DIAG:DEST:ACTIVE_ION_SOURCE': 2.0,
 'LS1_WA01:HMR_D1156:LAVG_RD': -3.5765918528340994e-05,
 'LS1_WA01:HMR_D1156:PKAVG_RD': -3.617910052393447e-05,
 'LS1_WA01:HMR_D1156:STD_RD': 0.062130568843949914,
 'LS1_WA01:HMR_D1156:TRIP_LAVGTIME_CSET': 10.0,
 'LS1_WA01:HMR_D1156:TRIP_LAVGHI_CSET': 0.0,
 'LS1_WA01:HMR_D1156:TRIP_LAVGHIEN_CMD': 0.0,
 'LS1_WA01:HMR_D1156:TRIP_LAVGLO_CSET': 0.0,
 'LS1_WA01:HMR_D1156:TRIP_LAVGLOEN_CMD': 0.0,
 'LS1_WA01:HMR_D1156:TRIP_EN_

In [83]:
df_hmr

,Device,Average,Peak Avg.,1 MHz Std.,Avg. Time,LongAvg.Th(H),LongAvg.Th(H)?,LongAvg.Th(L),LongAvg.Th(L)?,10ms Th(H),...,1500us Th(L),1500us Th(L)?,150us Th(H),150us Th(H)?,150us Th(L),150us Th(L)?,15us Th(H),15us Th(H)?,15us Th(L),15us Th(L)?
0,LS1_WA01:HMR_D1156,-3.576592e-05,-3.617910e-05,0.062131,10.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
1,LS1_WA02:HMR_D1190,3.425528e-04,3.454725e-04,0.051221,10.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
2,LS1_WA03:HMR_D1224,-4.034724e-06,-1.373703e-05,0.049812,10.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
3,LS1_WB01:HMR_D1287,-2.365273e-06,-6.537905e-06,0.058851,10.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
4,LS1_WB02:HMR_D1351,-1.307243e-05,-8.749513e-06,0.045233,10.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
5,LS1_WB03:HMR_D1415,2.649398e-05,2.651589e-05,0.037864,10.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
6,LS1_WB04:HMR_D1478,-1.416724e-06,9.563177e-06,0.029194,10.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
7,LS1_WB05:HMR_D1542,2.355098e-06,3.588062e-06,0.023953,10.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
8,LS1_WB06:HMR_D1606,-2.639400e-06,-5.728487e-06,0.019211,10.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
9,LS1_WB07:HMR_D1670,4.562700e-05,3.598451e-05,0.021593,10.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0


In [84]:
make_snapshot(['HMR'], data_dict, f"Generated with archived data on {datetime.now().isoformat()[:-7]}",
              ['AA', 'HMR'], con)

## Close the database

In [9]:
con.close()

## END

In [19]:
from phantasy_ui.widgets.model.utils import db_delete
# db_delete(con, data_dict['timestamp'], 'mps_threshold')

In [20]:
from phantasy_ui.widgets import BeamSpeciesDisplayWidget

In [21]:
BeamSpeciesDisplayWidget.get_species_meta_full()

['ISRC2',
 'Ca',
 48,
 20,
 10,
 20.0,
 1738.2655083036593,
 197.04705504699234,
 'LINAC',
 'Beam Delivered to Experiment']

In [22]:
from phantasy_apps.threshold_manager.data import SnapshotData

In [27]:
def make_snp(dtypes, data, note, tags):
    ts = data['timestamp']                                                                                              
    user = data['user']
    isrc_name, ion_name, ion_mass, ion_number, ion_charge, ion_charge_state, \
        beam_power, beam_energy, beam_bound, beam_dest = data['isrc_name'], data['ion_name'], \
        data['ion_mass'], data['ion_number'], data['ion_charge'], data['ion_charge_state'], \
        data['power'], data['energy'], data['bound'], 'N/A'                                                         
    data_dict = {_dtype: _get_dataframe_(_dtype, data) for _dtype in dtypes}                                            
    _row_data = (ts, user, isrc_name, ion_name, ion_number, ion_mass, ion_charge,                                       
                 ion_charge_state, beam_power, beam_energy, beam_bound, beam_dest,
                 ','.join(tags), note, data_dict)                                                                       
    snp_data = SnapshotData(_row_data)
    return snp_data

In [28]:
snp = make_snp(['ND'], data_dict, "Created with archived data", ['AA', 'ND'])

In [34]:
blob = snp.to_blob()

In [35]:
blob

b'PK\x03\x04\x14\x00\x00\x00\x08\x00NH\xf2V\x07AMb\x81\x00\x00\x00\xb1\x00\x00\x00\x10\x00\x00\x00docProps/app.xmlM\x8e=\x0b\x021\x10D\xff\xcaq\xbd\xb7A\xc1Bb@\xd0R\xb0\xb2\x0f{\x1b/\x90dC\xb2B~\xbe9\xc1\x8fn\x1eo\x18F\xdf\ng*\xe2\xa9\x0e-\x86T\x8f\xe3"\x92\x0f\x00\x15\x17\x8a\xb6N]\xa7n\x1c\x97h\xa5cy\x00;\xe7\x91\xce\x8c\xcfHI`\xab\xd4\x1e\xa8\t\xa5\x99\xe6M\xfe\x0e\x8eF\x9fr\x0e\x1e\xadxN\xe6\xea\xb1pe\'\xc3\xa5!\x05\r\xffrm\xde\xa9\xd45\xef&\xf5\x96\x1f\xd6\xf0;i^PK\x03\x04\x14\x00\x00\x00\x08\x00NH\xf2Vq\xbbub\xee\x00\x00\x00+\x02\x00\x00\x11\x00\x00\x00docProps/core.xml\xcd\x92\xcfj\xc30\x0c\x87_e\xf8\x9e(v\xa0\x01\x93\xe6\xd2\xb1\xd3\x06\x83\x156v3\xb6\xda\x9a\xc5\x7f\xb05\x92\xbe\xfd\x92\xacM\x19\xdb\x03\xech\xe9\xe7O\x9f@\xad\x8eR\x87\x84\xcf)DLd1\xdf\x8d\xae\xf7Y\xea\xb8e\'\xa2(\x01\xb2>\xa1S\xb9\x9c\x12~j\x1eBr\x8a\xa6g:BT\xfaC\x1d\x11DUm\xc0!)\xa3H\xc1\x0c,\xe2Jd]k\xb4\xd4\t\x15\x85t\xc1\x1b\xbd\xe2\xe3g\xea\x17\x98\xd1\x80=:\xf4\x94\x81\x97\x1cX7O\x8c\xe7\xb1o\xe1\x06\x98a

In [36]:
df_dict = SnapshotData.read_blob(blob)

In [39]:
df_dict.keys()

dict_keys(['info', 'ND'])

In [40]:
df_info = df_dict.get('info')

In [41]:
df_info

,timestamp,1681267620
0,datetime,2023-04-11T22:47:00
1,user,zhangt
2,isrc_name,ISRC2
3,ion_name,Xe
4,ion_number,54
5,ion_mass,124
6,ion_charge,26
7,ion_charge_state,34
8,beam_power,4908.69
9,beam_energy,226.963


In [42]:
df_nd = df_dict.get('ND')

In [43]:
df_nd

,Device,Average,Peak Avg.,1 MHz Std.,Avg. Time,LongAvg.Th(H),LongAvg.Th(H)?,LongAvg.Th(L),LongAvg.Th(L)?,10ms Th(H),...,1500us Th(L),1500us Th(L)?,150us Th(H),150us Th(H)?,150us Th(L),150us Th(L)?,15us Th(H),15us Th(H)?,15us Th(L),15us Th(L)?
0,FE_RFQ:ND_D1025,-0.000088,-0.000047,0.001349,1,0,0,-0.041873,1,0.0000,...,-27.905400,1,0,0,0,0,0,0,0,0
1,LS1_CA01:ND_D1128,-0.003611,-0.005193,0.044035,1,0,0,-0.053538,1,0.0000,...,-27.935310,1,0,0,0,0,0,0,0,0
2,LS1_CA03:ND_D1195,-0.015497,-0.013918,0.157742,1,0,0,-0.042471,1,0.0000,...,-27.905400,1,0,0,0,0,0,0,0,0
3,LS1_CB01:ND_D1250,-0.012084,-0.006480,0.124692,1,0,0,-0.041873,1,0.0000,...,-27.905400,1,0,0,0,0,0,0,0,0
4,LS1_CB03:ND_D1378,-0.001209,-0.000495,0.019865,1,0,0,-0.042770,1,0.0000,...,-27.815670,1,0,0,0,0,0,0,0,0
5,LS1_CB05:ND_D1505,-0.000765,-0.000854,0.016387,1,0,0,-0.041873,1,0.0000,...,-27.815670,1,0,0,0,0,0,0,0,0
6,LS1_CB07:ND_D1633,-0.000356,-0.003745,0.011963,1,0,0,-0.041873,1,0.0000,...,-27.815670,1,0,0,0,0,0,0,0,0
7,LS1_CB09:ND_D1760,-0.000244,-0.000170,0.013447,1,0,0,-0.041873,1,0.0000,...,-27.815670,1,0,0,0,0,0,0,0,0
8,LS1_CB11:ND_D1908,-0.000529,-0.000246,0.022898,1,0,0,-0.041873,1,0.0000,...,-27.815670,1,0,0,0,0,0,0,0,0
9,LS1_BTS:ND_D2076,-0.008347,-0.014829,0.115892,1,0,0,-0.042471,1,0.0000,...,-27.815670,1,0,0,0,0,0,0,0,0
